## Set Data Paths (Action Needed)

In [3]:
from pathlib import Path
# Please set data_path to the location that contains 01302019.NASDAQ_ITCH50
data_path = Path('D:/') 
itch_store = data_path / 'itch.h5'
order_book_store = data_path / 'order_book.h5'
vwap_store = data_path / 'vwap_store.h5'
print('Setting data paths')

In [ ]:
ITCH_filename='01302019.NASDAQ_ITCH50'

## Import Libraries

In [1]:
print('Importing libraries')
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import timedelta
from time import time
from struct import unpack
from collections import namedtuple, Counter

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Part1: Extract Data from NASDAQ_ITCH50
## ITCH Format Settings

In [ ]:
print('Part1: Extract Data from NASDAQ_ITCH50')
print('Setting ITCH Format')
# System Event Codes – Daily
event_codes = {'O': 'Start of Messages',
               'S': 'Start of System Hours',
               'Q': 'Start of Market Hours',
               'M': 'End of Market Hours',
               'E': 'End of System Hours',
               'C': 'End of Messages'}

encoding = {'primary_market_maker': {'Y': 1, 'N': 0},
            'printable'           : {'Y': 1, 'N': 0},
            'buy_sell_indicator'  : {'B': 1, 'S': -1},
            'cross_type'          : {'O': 0, 'C': 1, 'H': 2},         # “O” = Opening Cross, “C” = Closing Cross, 
                                                                      # “H” = Cross for IPO and halted / paused securities
            'imbalance_direction' : {'B': 0, 'S': 1, 'N': 0, 'O': -1}}
                                                                      # “B” = buy imbalance, “S” = sell imbalance
                                                                      # “N” = no imbalance, “O” = Insufficient orders to calculate

# Formats Dictionary
# It is used to assemble the format strings
formats = { ('integer', 2): 'H',   # int of length 2 => format string 'H'
            ('integer', 4): 'I',
            ('integer', 6): '6s',  # int of length 6 => parse as string, convert later
            ('integer', 8): 'Q',
            ('alpha', 1)  : 's',
            ('alpha', 2)  : '2s',
            ('alpha', 4)  : '4s',
            ('alpha', 8)  : '8s',
            ('price_4', 4): 'I',
            ('price_8', 8): 'Q', }

## Cleaning message_types.xlsx and Creating Data Format

In [ ]:
print('Processing data format for Part 1')
print('Cleaning message_types.xlsx')
def clean_message_types(df):
    # transfer all the columns' name into lower case and remove all the leading and trailing spaces
    df.columns = [c.lower().strip() for c in df.columns]
    # remove all the leading and trailing spaces in the values
    df.value = df.value.str.strip()
    # clean all values in the name column 
    df.name = (df.name
               .str.strip()
               .str.lower()
               .str.replace(' ', '_')
               .str.replace('-', '_')
               .str.replace('/', '_'))
    # clean all values in the notes column 
    df.notes = df.notes.str.strip()
    # add message type column
    df['message_type'] = df.loc[df.name == 'message_type', 'value']
    return df

#### Load Message Types

In [ ]:
print('Processing message_types.xlsx')
message_types = clean_message_types(pd.read_excel('message_types.xlsx', sheet_name='messages', encoding='latin1')
                                    .sort_values('id').drop('id', axis=1))

# Get Message Labels
print('Creating message_labels')
# extract the notes and message type columns that has value
message_labels = (message_types.loc[:, ['message_type', 'notes']]
                  .dropna()
                  .rename(columns={'notes': 'name'}))

# clean the notes column
message_labels.name = (message_labels.name
                       .str.lower()
                       .str.replace('message', '')
                       .str.replace('.', '')
                       .str.strip().str.replace(' ', '_'))

# message_labels.to_csv('message_labels.csv', index=False)

# fill NA in the message_type column
message_types.message_type = message_types.message_type.ffill()

# remove the message type in the rows
message_types = message_types[message_types.name != 'message_type']

# clean the value column in the data frame
message_types.value = (message_types.value
                       .str.lower()
                       .str.replace(' ', '_')
                       .str.replace('(', '')
                       .str.replace(')', ''))

message_types.info()

#### Get message specification

In [ ]:
print('Creating ITCH specs')
# Get ITCH specs and create formatting (type, length) tuples
specs = message_types
specs['formats'] = specs[['value', 'length']].apply(tuple, axis=1).map(formats)

#### Extract alpha format 

In [ ]:
print('Extracting alpha format')
# Extract formatting for alpha numerical fields
alpha_fields = specs[specs.value == 'alpha'].set_index('name')
# Groupby the message types
alpha_msgs = alpha_fields.groupby('message_type')
# Create the alpha format dictionary 
alpha_formats = {k: v.to_dict() for k, v in alpha_msgs.formats}
# Create the alpha length dictionary
alpha_length = {k: v.add(5).to_dict() for k, v in alpha_msgs.length}

#### Generate message classes as named tuples and format strings

In [ ]:
message_fields, fstring = {}, {}
for t, message in specs.groupby('message_type'):
    message_fields[t] = namedtuple(typename=t, field_names=message.name.tolist())
    fstring[t] = '>' + ''.join(message.formats.tolist())

## Functions for Processing Data from ITCH 5.0

In [ ]:
def format_alpha(mtype, data):
    """Process byte strings of type alpha"""
    for col in alpha_formats.get(mtype).keys():
        if mtype != 'R' and col == 'stock':
            data = data.drop(col, axis=1)
            continue
        data.loc[:, col] = data.loc[:, col].str.decode("utf-8").str.strip()
        if encoding.get(col):
            data.loc[:, col] = data.loc[:, col].map(encoding.get(col))
    return data

def store_messages(m):
    """Handle occasional storing of all messages"""
    with pd.HDFStore(itch_store) as store:
        for mtype, data in m.items():
            # convert to DataFrame
            data = pd.DataFrame(data)

            # parse timestamp info
            data.timestamp = data.timestamp.apply(int.from_bytes, byteorder='big')
            data.timestamp = pd.to_timedelta(data.timestamp)

            # apply alpha formatting
            if mtype in alpha_formats.keys():
                data = format_alpha(mtype, data)

            s = alpha_length.get(mtype)
            if s:
                s = {c: s.get(c) for c in data.columns}
            dc = ['stock_locate']
            if m == 'R':
                dc.append('stock')
            store.append(mtype,
                         data,
                         format='t',
                         min_itemsize=20,  # set to a relative large value to deal with the length of data
                         data_columns=dc)

## Extract Data from ITCH50

In [ ]:
print('Extracting Data from ITCH50')
print('About 30 mins...')
start = time()
messages = {}
message_count = 0
message_type_counter = Counter()
data  = open(data_path / ITCH_filename, mode='rb')
while True:

    # Obtain the size of message in bytes
    message_size = int.from_bytes(data.read(2), byteorder='big', signed=False)

    # Get message type by reading first byte
    message_type = data.read(1).decode('ascii')

    # Create data structure to capture result
    if not messages.get(message_type):
        messages[message_type] = []

    message_type_counter.update([message_type])

    # Read & store the rest of one message
    record = data.read(message_size - 1)
    message = message_fields[message_type]._make(unpack(fstring[message_type], record))
    messages[message_type].append(message)

    # deal with system events
    if message_type == 'S':
        timestamp = int.from_bytes(message.timestamp, byteorder='big')
        print('\n', event_codes.get(message.event_code.decode('ascii'), 'Error'))
        print('\t{0}\t{1:,.0f}'.format(timedelta(seconds=timestamp * 1e-9),
                                     message_count))
        if message.event_code.decode('ascii') == 'C':
            store_messages(messages)
            break

    message_count += 1
    if message_count % 2.5e7 == 0:
        timestamp = int.from_bytes(message.timestamp, byteorder='big')
        print('\t{0}\t{1:,.0f}\t{2}'.format(timedelta(seconds=timestamp * 1e-9),
                                            message_count,
                                            timedelta(seconds=time() - start)))
        store_messages(messages)
        messages = {}

print(timedelta(seconds=time() - start))

# Part 2: Build Order Book and Calculate VWAP

In [4]:
date = '01302019'
date = ITCH_filename.split('.')[0]
date = '20190130'
date = str(pd.to_datetime(date, format='%m%d%Y').date()).replace('-','')

## Functions for Building Order Book

In [5]:
def get_trades(m):
    """Combine C, E, P and Q messages into trading records"""
    trade_dict = {'executed_shares': 'shares', 'execution_price': 'price'}
    cols = ['timestamp', 'executed_shares']
    trades = pd.concat([m.loc[m.type == 'E', cols + ['price']].rename(columns=trade_dict),
                        m.loc[m.type == 'C', cols + ['execution_price']].rename(columns=trade_dict),
                        m.loc[m.type == 'P', ['timestamp', 'price', 'shares']],
                        m.loc[m.type == 'Q', ['timestamp', 'price', 'shares']].assign(cross=1),
                        ], sort=False).dropna(subset=['price']).fillna(0)
    return trades.set_index('timestamp').sort_index().astype(int)

def add_orders(orders, buysell, nlevels):
    """Add orders up to desired depth given by nlevels;
        sell in ascending, buy in descending order
    """
    new_order = []
    items = sorted(orders.copy().items())
    if buysell == 1:
        items = reversed(items)  
    for i, (p, s) in enumerate(items, 1):
        new_order.append((p, s))
        if i == nlevels:
            break
    return orders, new_order

def save_orders(orders, append=False):
    cols = ['price', 'shares']
    for buysell, book in orders.items():
        df = (pd.concat([pd.DataFrame(data=data,
                                     columns=cols)
                         .assign(timestamp=t) 
                         for t, data in book.items()]))
        key = '{}/{}'.format(stock, order_dict[buysell])
        df.loc[:, ['price', 'shares']] = df.loc[:, ['price', 'shares']].astype(int)
        with pd.HDFStore(order_book_store) as store:
            if append:
                store.append(key, df.set_index('timestamp'), format='t')
            else:
                store.put(key, df.set_index('timestamp'))
                
def vwap(agg_trades):
    vwap = agg_trades.apply(lambda x: np.average(x.price, weights=x.shares)).to_frame('vwap')
    return vwap
order_dict = {-1: 'sell', 1: 'buy'}

In [8]:
start = time()
stock_list=list(set(pd.HDFStore(itch_store).select('R')['stock']))
stock_list.sort()
for i in range(len(stock_list)):
    stock = stock_list[i]
    print(stock+' '+str(round(i/float(len(stock_list))*100, 2))+'% Completed', end='\r')
    date = '01302019'
    def get_messages(date, stock=stock):
        """Collect trading messages for given stock"""
        with pd.HDFStore(itch_store) as store:
            stock_locate = store.select('R')[store.select('R')['stock']==stock]['stock_locate']
            target = 'stock_locate = stock_locate'

            data = {}
            # trading message types
            messages = ['A', 'F', 'E', 'C', 'X', 'D', 'U', 'P', 'Q']
            for m in messages:
                data[m] = store.select(m, where=target).drop('stock_locate', axis=1).assign(type=m)

        order_cols = ['order_reference_number', 'buy_sell_indicator', 'shares', 'price']
        orders = pd.concat([data['A'], data['F']], sort=False, ignore_index=True).loc[:, order_cols]

        for m in messages[2: -3]:
            data[m] = data[m].merge(orders, how='left')

        data['U'] = data['U'].merge(orders, how='left',
                                    right_on='order_reference_number',
                                    left_on='original_order_reference_number',
                                    suffixes=['', '_replaced'])

        data['Q'].rename(columns={'cross_price': 'price'}, inplace=True)
        data['X']['shares'] = data['X']['cancelled_shares']
        data['X'] = data['X'].dropna(subset=['price'])

        data = pd.concat([data[m] for m in messages], ignore_index=True, sort=False)
        data['date'] = pd.to_datetime(date, format='%m%d%Y')
        data.timestamp = data['date'].add(data.timestamp)
        data = data[data.printable != 0]

        drop_cols = ['tracking_number', 'order_reference_number', 'original_order_reference_number',
                     'cross_type', 'new_order_reference_number', 'attribution', 'match_number',
                     'printable', 'date', 'cancelled_shares']
        return data.drop(drop_cols, axis=1).sort_values('timestamp').reset_index(drop=True)
    messages = get_messages(date=date, stock=stock)
    with pd.HDFStore(order_book_store) as store:
        key = '{}/messages'.format(stock)
        store.put(key, messages)
    trades = get_trades(messages)
    with pd.HDFStore(order_book_store) as store:
        store.put('{}/trades'.format(stock), trades)
    
    order_book = {-1: {}, 1: {}}
    current_orders = {-1: Counter(), 1: Counter()}
    message_counter = Counter()
    nlevels = 100

    start = time()
    for message in messages.itertuples():
        i = message[0]
        if i % 1e5 == 0 and i > 0:
            #print('{:,.0f}\t\t{}'.format(i, timedelta(seconds=time() - start)))
            save_orders(order_book, append=True)
            order_book = {-1: {}, 1: {}}
            start = time()
        if np.isnan(message.buy_sell_indicator):
            continue
        message_counter.update(message.type)

        buysell = message.buy_sell_indicator
        price, shares = None, None

        if message.type in ['A', 'F', 'U']:
            price = int(message.price)
            shares = int(message.shares)

            current_orders[buysell].update({price: shares})
            current_orders[buysell], new_order = add_orders(current_orders[buysell], buysell, nlevels)
            order_book[buysell][message.timestamp] = new_order

        if message.type in ['E', 'C', 'X', 'D', 'U']:
            if message.type == 'U':
                if not np.isnan(message.shares_replaced):
                    price = int(message.price_replaced)
                    shares = -int(message.shares_replaced)
            else:
                if not np.isnan(message.price):
                    price = int(message.price)
                    shares = -int(message.shares)

            if price is not None:
                current_orders[buysell].update({price: shares})
                if current_orders[buysell][price] <= 0:
                    current_orders[buysell].pop(price)
                current_orders[buysell], new_order = add_orders(current_orders[buysell], buysell, nlevels)
                order_book[buysell][message.timestamp] = new_order
    date = '20180130'
    with pd.HDFStore(itch_store) as store:
        sys_events = store['S'].set_index('event_code').drop_duplicates()
        sys_events.timestamp = sys_events.timestamp.add(pd.to_datetime(date)).dt.time
        market_open = sys_events.loc['Q', 'timestamp']
        market_close = sys_events.loc['M', 'timestamp']
    with pd.HDFStore(order_book_store) as store:
        trades = store['{}/trades'.format(stock)]

    trades.price = trades.price.mul(1e-4)
    trades = trades[trades.cross == 0]
    trades = trades.between_time(market_open, market_close).drop('cross', axis=1)
    trades_per_min = trades.shares.sum()/(60*7.5) # min per trading day
    trades['cumul_vol'] = trades.shares.cumsum()
    if trades.shape[0] == 0:
        empty_output=pd.DataFrame([['Not Available', 'Not Available']], columns=['timestamp', 'vwap'])
        with pd.HDFStore(vwap_store) as store:
            key = stock
            store.put(key, empty_output)
        continue
    df = trades.reset_index()
    by_vol = df.groupby(df.cumul_vol.div(trades_per_min).round().astype(int))
    vwap_time = pd.concat([by_vol.timestamp.last().to_frame('timestamp'), vwap(by_vol)], axis=1)
    vwap_time.index.names=[''] 
    with pd.HDFStore(vwap_store) as store:
        key = stock
        store.put(key, vwap_time)
print('The total time for creating VWAP is '+str(timedelta(seconds=time() - start))+'.')

C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'ABR-A'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'ABR-B'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'ABR-C'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'AFSI-A'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'AFSI-B'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


...

C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'AMH-E'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'AMH-F'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'AMH-G'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


C:\Users\KevinZhang\Anaconda3\lib\site-packages\tables\path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'AMH-H'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


KeyboardInterrupt: 

In [12]:
pd.HDFStore(vwap_store).select('AMRK')

,timestamp,vwap
,,
1,2019-01-30 09:30:31.260658008,13.240000
3,2019-01-30 09:31:57.058936927,13.240000
5,2019-01-30 11:16:20.340411396,13.170000
6,2019-01-30 11:43:42.569926209,13.090000
25,2019-01-30 12:42:17.685297852,13.140000
26,2019-01-30 12:42:28.384250679,13.100000
27,2019-01-30 12:42:28.384250679,13.100000
28,2019-01-30 12:45:51.019992372,13.150000
47,2019-01-30 12:49:23.672321572,13.240000


In [14]:
['A','AA','AAAU','AABA','AAC','AADR','AAL','AAMC','AAME','AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'AAU', 'AAWW', 'AAXJ', 'AAXN', 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABDC',
 'ABEO',
 'ABEOW',
 'ABEV',
 'ABG',
 'ABIL',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABR-A',
 'ABR-B',
 'ABR-C',
 'ABT',
 'ABTX',
 'ABUS',
 'AC',
 'ACA',
 'ACAD',
 'ACB',
 'ACBI',
 'ACC',
 'ACCO',
 'ACER',
 'ACES',
 'ACET',
 'ACGL',
 'ACGLO',
 'ACGLP',
 'ACH',
 'ACHC',
 'ACHN',
 'ACHV',
 'ACIA',
 'ACIM',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACSG',
 'ACSI',
 'ACST',
 'ACT',
 'ACTG',
 'ACU',
 'ACV',
 'ACWF',
 'ACWI',
 'ACWV',
 'ACWX',
 'ACY',
 'ADAP',
 'ADBE',
 'ADC',
 'ADES',
 'ADI',
 'ADIL',
 'ADILW',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADNT',
 'ADOM',
 'ADP',
 'ADRA',
 'ADRD',
 'ADRE',
 'ADRO',
 'ADRU',
 'ADS',
 'ADSK',
 'ADSW',
 'ADT',
 'ADTN',
 'ADUS',
 'ADVM',
 'ADX',
 'ADXS',
 'AE',
 'AEB',
 'AED',
 'AEE',
 'AEF',
 'AEG',
 'AEGN',
 'AEH',
 'AEHR',
 'AEIS',
 'AEL',
 'AEM',
 'AEMD',
 'AEO',
 'AEP',
 'AER',
 'AERI',
 'AES',
 'AETI',
 'AEY',
 'AEYE',
 'AEZS',
 'AFB',
 'AFC',
 'AFG',
 'AFGE',
 'AFGH',
 'AFH',
 'AFHBL',
 'AFI',
 'AFIF',
 'AFIN',
 'AFK',
 'AFL',
 'AFMD',
 'AFSI-A',
 'AFSI-B',
 'AFSI-C',
 'AFSI-D',
 'AFSI-E',
 'AFSI-F',
 'AFSS',
 'AFST',
 'AFT',
 'AFTY',
 'AG',
 'AGCO',
 'AGD',
 'AGE',
 'AGEN',
 'AGF',
 'AGFS',
 'AGFSW',
 'AGG',
 'AGGE',
 'AGGP',
 'AGGY',
 'AGI',
 'AGIO',
 'AGLE',
 'AGM',
 'AGM-A',
 'AGM-B',
 'AGM-C',
 'AGM.A',
 'AGMH',
 'AGN',
 'AGNC',
 'AGNCB',
 'AGNCN',
 'AGND',
 'AGO',
 'AGO-B',
 'AGO-E',
 'AGO-F',
 'AGQ',
 'AGR',
 'AGRO',
 'AGRX',
 'AGS',
 'AGT',
 'AGTC',
 'AGX',
 'AGYS',
 'AGZ',
 'AGZD',
 'AHC',
 'AHH',
 'AHL',
 'AHL-C',
 'AHL-D',
 'AHPI',
 'AHT',
 'AHT-D',
 'AHT-F',
 'AHT-G',
 'AHT-H',
 'AHT-I',
 'AI',
 'AI-B',
 'AIA',
 'AIC',
 'AIEQ',
 'AIF',
 'AIG',
 'AIG+',
 'AIHS',
 'AIIQ',
 'AIMC',
 'AIMT',
 'AIN',
 'AINC',
 'AINV',
 'AIPT',
 'AIQ',
 'AIR',
 'AIRG',
 'AIRI',
 'AIRR',
 'AIRT',
 'AIT',
 'AIV',
 'AIV-A',
 'AIW',
 'AIY',
 'AIZ',
 'AIZP',
 'AJG',
 'AJRD',
 'AJX',
 'AJXA',
 'AKAM',
 'AKAO',
 'AKBA',
 'AKCA',
 'AKER',
 'AKG',
 'AKO.A',
 'AKO.B',
 'AKP',
 'AKR',
 'AKRX',
 'AKS',
 'AKTS',
 'AKTX',
 'AL',
 'ALAC',
 'ALACR',
 'ALACU',
 'ALACW',
 'ALB',
 'ALBO',
 'ALCO',
 'ALD',
 'ALDR',
 'ALDX',
 'ALE',
 'ALEX',
 'ALFA',
 'ALG',
 'ALGN',
 'ALGR',
 'ALGRR',
 'ALGRU',
 'ALGRW',
 'ALGT',
 'ALIM',
 'ALJJ',
 'ALK',
 'ALKS',
 'ALL',
 'ALL-A',
 'ALL-B',
 'ALL-D',
 'ALL-E',
 'ALL-F',
 'ALL-G',
 'ALLE',
 'ALLK',
 'ALLO',
 'ALLT',
 'ALLY',
 'ALLY-A',
 'ALNA',
 'ALNY',
 'ALO',
 'ALOT',
 'ALP-Q',
 'ALPN',
 'ALQA',
 'ALRM',
 'ALRN',
 'ALSK',
 'ALSN',
 'ALT',
 'ALTM',
 'ALTR',
 'ALTS',
 'ALTY',
 'ALV',
 'ALX',
 'ALXN',
 'ALYA',
 'AM',
 'AMAG',
 'AMAL',
 'AMAT',
 'AMBA',
 'AMBC',
 'AMBCW',
 'AMBO',
 'AMBR',
 'AMC',
 'AMCA',
 'AMCI',
 'AMCIU',
 'AMCIW',
 'AMCN',
 'AMCX',
 'AMD',
 'AME',
 'AMED',
 'AMEH',
 'AMG',
 'AMGN',
 'AMGP',
 'AMH',
 'AMH-D',
 'AMH-E',
 'AMH-F',
 'AMH-G',
 'AMH-H',
 'AMID',
 'AMJ',
 'AMJL',
 'AMKR',
 'AMLP',
 'AMMA',
 'AMN',
 'AMNB',
 'AMOT',
 'AMOV',
 'AMP',
 'AMPE',
 'AMPH',
 'AMR',
 'AMRB',
 'AMRC',
 'AMRH',
 'AMRHW',
 'AMRK']

['A',
 'AA',
 'AAAU',
 'AABA',
 'AAC',
 'AADR',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AAWW',
 'AAXJ',
 'AAXN',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABDC',
 'ABEO',
 'ABEOW',
 'ABEV',
 'ABG',
 'ABIL',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABR-A',
 'ABR-B',
 'ABR-C',
 'ABT',
 'ABTX',
 'ABUS',
 'AC',
 'ACA',
 'ACAD',
 'ACB',
 'ACBI',
 'ACC',
 'ACCO',
 'ACER',
 'ACES',
 'ACET',
 'ACGL',
 'ACGLO',
 'ACGLP',
 'ACH',
 'ACHC',
 'ACHN',
 'ACHV',
 'ACIA',
 'ACIM',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACSG',
 'ACSI',
 'ACST',
 'ACT',
 'ACTG',
 'ACU',
 'ACV',
 'ACWF',
 'ACWI',
 'ACWV',
 'ACWX',
 'ACY',
 'ADAP',
 'ADBE',
 'ADC',
 'ADES',
 'ADI',
 'ADIL',
 'ADILW',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADNT',
 'ADOM',
 'ADP',
 'ADRA',
 'ADRD',
 'ADRE',
 'ADRO',
 'ADRU',
 'ADS',
 'ADSK',
 'ADSW',
 'ADT',
 'ADTN',
 'ADUS',
 'ADVM',
 'ADX',
 'ADXS',
 'AE',
 'AEB',
 'AED',
 'AEE',
 '

In [11]:
stock_list=list(set(pd.HDFStore(itch_store).select('R')['stock']))
stock_list.sort()
stock_list=stock_list
stock_list

['A',
 'AA',
 'AAAU',
 'AABA',
 'AAC',
 'AADR',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AAWW',
 'AAXJ',
 'AAXN',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABDC',
 'ABEO',
 'ABEOW',
 'ABEV',
 'ABG',
 'ABIL',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABR-A',
 'ABR-B',
 'ABR-C',
 'ABT',
 'ABTX',
 'ABUS',
 'AC',
 'ACA',
 'ACAD',
 'ACB',
 'ACBI',
 'ACC',
 'ACCO',
 'ACER',
 'ACES',
 'ACET',
 'ACGL',
 'ACGLO',
 'ACGLP',
 'ACH',
 'ACHC',
 'ACHN',
 'ACHV',
 'ACIA',
 'ACIM',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACSG',
 'ACSI',
 'ACST',
 'ACT',
 'ACTG',
 'ACU',
 'ACV',
 'ACWF',
 'ACWI',
 'ACWV',
 'ACWX',
 'ACY',
 'ADAP',
 'ADBE',
 'ADC',
 'ADES',
 'ADI',
 'ADIL',
 'ADILW',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADNT',
 'ADOM',
 'ADP',
 'ADRA',
 'ADRD',
 'ADRE',
 'ADRO',
 'ADRU',
 'ADS',
 'ADSK',
 'ADSW',
 'ADT',
 'ADTN',
 'ADUS',
 'ADVM',
 'ADX',
 'ADXS',
 'AE',
 'AEB',
 'AED',
 'AEE',
 '

In [ ]:
#store=pd.HDFStore(vwap_store)
#store.append(stock, vwap_time, append=True, format='t')
with pd.HDFStore(vwap_store) as store:
        key = '{}/messages'.format(stock)
        store.put(key, vwap_time)

In [ ]:
type(vwap_time)

In [ ]:
key = stock
store=pd.HDFStore(vwap_store)
store.put(key, vwap_time)

In [ ]:
vwap_time

In [ ]:
getattr(store, stock)

In [ ]:
stock

In [ ]:
pd.HDFStore(vwap_store).select('ALL-G')